In [1]:
# # install octave
# !sudo apt-get -qq update
# !sudo apt-get -qq install octave octave-signal liboctave-dev

# # install oct2py that compatible with colab
# import os

# from pkg_resources import get_distribution

# os.system(
#     f"pip install -qq"
#     f" ipykernel=={get_distribution('ipykernel').version}"
#     f" ipython=={get_distribution('ipython').version}"
#     f" tornado=={get_distribution('tornado').version}"
#     f" oct2py"
# )

# # install packages
# !pip install -qq matpower matpowercaseframes

In [2]:
import matpower
import oct2py

print(f"oct2py version: {oct2py.__version__}")
print(f"matpower version: {matpower.__version__}")

oct2py version: 5.8.0
matpower version: 8.1.0.2.2.3


In [3]:
import os

import pandas as pd
from matpower import path_matpower, start_instance

In [7]:
path_t_cases = os.path.join(path_matpower, "lib/t")

In [6]:
m = start_instance()

In [8]:
# load single most example case
mpc = m.loadcase(os.path.join(path_t_cases, "t_case30_userfcns.m"))
mpc

{'version': '2',
 'baseMVA': 100.0,
 'bus': array([[1.00e+00, 3.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         1.00e+00, 1.00e+00, 0.00e+00, 1.35e+02, 1.00e+00, 1.05e+00,
         9.50e-01],
        [2.00e+00, 2.00e+00, 2.17e+01, 1.27e+01, 0.00e+00, 0.00e+00,
         1.00e+00, 1.00e+00, 0.00e+00, 1.35e+02, 1.00e+00, 1.10e+00,
         9.50e-01],
        [3.00e+02, 1.00e+00, 2.40e+00, 1.20e+00, 0.00e+00, 0.00e+00,
         1.00e+00, 1.00e+00, 0.00e+00, 1.35e+02, 1.00e+00, 1.05e+00,
         9.50e-01],
        [4.00e+00, 1.00e+00, 7.60e+00, 1.60e+00, 0.00e+00, 0.00e+00,
         1.00e+00, 1.00e+00, 0.00e+00, 1.35e+02, 1.00e+00, 1.05e+00,
         9.50e-01],
        [5.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 1.90e-01,
         1.00e+00, 1.00e+00, 0.00e+00, 1.35e+02, 1.00e+00, 1.05e+00,
         9.50e-01],
        [6.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         1.00e+00, 1.00e+00, 0.00e+00, 1.35e+02, 1.00e+00, 1.05e+00,
         9.50e-01],
        [

In [9]:
mpc.keys()

dict_keys(['version', 'baseMVA', 'bus', 'gen', 'branch', 'gencost', 'reserves', 'if'])

In [10]:
from matpowercaseframes.constants import COLUMNS

In [11]:
COLUMNS.keys()

dict_keys(['bus', 'gen', 'branch', 'dcline', 'if', 'gencost', 'dclinecost', 'bus_name', 'branch_name', 'gen_name', 'case'])

In [22]:
mpc_keys = set(mpc.keys())
columns_keys = set(COLUMNS.keys())
missing_keys = mpc_keys - columns_keys
missing_keys

{'baseMVA', 'reserves', 'version'}

In [23]:
mpc.reserves

{'zones': array([[1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.]]),
 'req': array([[60.],
        [20.]]),
 'cost': array([[1.9],
        [2. ],
        [3. ],
        [4. ],
        [5. ],
        [5.5]]),
 'qty': array([[25.],
        [25.],
        [25.],
        [25.],
        [25.],
        [25.]]),
 'getdoc': {}}

Reserve Data
reserve zones, element i, j is 1 if gen j is in zone i, 0 otherwise.
dimension (zone, gen)

```octave
mpc.reserves.zones = [
	1	1	1	0;
];
```

Mapped to `DataFrame` with index follows zone index (from 1), and columns follows
generator index (cf.gen.index).

In [24]:
reserves_zone = mpc.reserves.zones
n_zones, n_gens = reserves_zone.shape
df_reserves_zone = pd.DataFrame(
    reserves_zone,
    index=pd.RangeIndex(start=1, stop=n_zones + 1, name="zone"),
    columns=pd.RangeIndex(start=1, stop=n_gens + 1, name="gen"),
)
df_reserves_zone

gen,1,2,3,4,5,6
zone,,,,,,
1,1.0,1.0,1.0,1.0,1.0,1.0
2,0.0,0.0,0.0,0.0,1.0,1.0


reserve requirements for each zone in MW

```octave
mpc.reserves.req = [60; 20];
```

or

```octave
mpc.reserves.req = 150;
```

Mapped to `DataFrame` with index follows zone index (from 1), and single column 'PREQ'.

In [25]:
reserves_req = mpc.reserves.req
df_reserves_req = pd.DataFrame(
    reserves_req,
    index=pd.RangeIndex(start=1, stop=n_zones + 1, name="zone"),
    columns=["PREQ"],
)
df_reserves_req

,PREQ
zone,
1,60.0
2,20.0


In [37]:
df_reserves_zone_sum = df_reserves_zone.sum(axis=0)
idx_gen_with_reserves = df_reserves_zone_sum[df_reserves_zone_sum > 0].index
idx_gen_with_reserves

RangeIndex(start=1, stop=7, step=1, name='gen')

In [39]:
reserves_cost = mpc.reserves.cost
df_reserves_cost = pd.DataFrame(
    reserves_cost, index=idx_gen_with_reserves, columns=["C1"]
)
df_reserves_cost

,C1
gen,
1,1.9
2,2.0
3,3.0
4,4.0
5,5.0
6,5.5


In [45]:
# TODO: mpc.if (need to use mpc['if'])
mpc["if"]

{'map': array([[  1., -12.],
        [  1., -14.],
        [  1., -15.],
        [  1., -36.],
        [  2.,  15.],
        [  2.,  25.],
        [  2.,  26.],
        [  2., -32.],
        [  3.,  12.],
        [  3.,  14.],
        [  3., -25.],
        [  3., -26.],
        [  3.,  32.],
        [  3.,  36.]]),
 'lims': array([[  1., -15.,  25.],
        [  2., -10.,  20.]])}

In [13]:
# m.exit()